# Set-Up

In [1]:
!pip install tetgen
!apt-get install -qq xvfb
!pip install pyvista panel -q
!pip install -q piglet pyvirtualdisplay
# !pip install panel

     |████████████████████████████████| 1.9 MB 2.6 MB/s eta 0:00:01     |██████████████████████          | 1.3 MB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 10.1 MB/s eta 0:00:01
     |████████████████████████████████| 59.5 MB 34 kB/s  eta 0:00:011   |████▊                           | 8.8 MB 8.6 MB/s eta 0:00:06     |██████▎                         | 11.6 MB 7.5 MB/s eta 0:00:07     |████████▎                       | 15.4 MB 7.5 MB/s eta 0:00:06     |██████████▌                     | 19.4 MB 8.6 MB/s eta 0:00:05     |█████████████████               | 31.8 MB 9.6 MB/s eta 0:00:03     |█████████████████▊              | 33.0 MB 9.6 MB/s eta 0:00:03     |███████████████████▍            | 36.1 MB 7.1 MB/s eta 0:00:04     |███████████████████▉            | 36.9 MB 7.1 MB/s eta 0:00:04     |███████████████████████         | 43.0 MB 7.4 MB/s eta 0:00:03
     |████████████████████████████████| 158 kB 10.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 9

In [2]:
# MUST RUN THIS CELL:
!pip uninstall -y h5py
!pip install h5py==2.9.0

     |████████████████████████████████| 287 kB 5.6 MB/s eta 0:00:01     |████████                        | 71 kB 6.6 MB/s eta 0:00:01
  Created wheel for h5py: filename=h5py-2.9.0-cp39-cp39-linux_x86_64.whl size=4691499 sha256=eef65538297b065516384e2f2681f53ab96b319d4c9d1f77c88ef4cb51c91618
  Stored in directory: /root/.cache/pip/wheels/ef/54/5c/3fbdb9cfe071661699815cfd6b71ddf2d12d61d121a109a5e0
Successfully built h5py


In [132]:
# Fenics imports:
import dolfinx
import dolfinx.io
import dolfinx.plot
from dolfinx.cpp.mesh import CellType

# Numerics imports:
from petsc4py import PETSc
import numpy as np

# Mesh imports:
import gmsh
import meshio
import ufl
from mpi4py import MPI
import tetgen

# Visualisation imports:
import pyvista

# Misc imports:
from math import sin, cos, pi, ceil, floor
import os

# Mesh Conversion

In [133]:
def tetgen_to_meshio(tetgen_mesh):
    pyvista.save_meshio('temp.msh', tetgen_mesh)
    mesh = meshio.read('temp.msh')
    os.remove('temp.msh')
    return mesh

def meshio_to_grid(meshio_mesh):
    topology, cell_types = dolfinx.plot.create_vtk_topology(meshio_mesh, meshio_mesh.topology.dim)
    grid = pyvista.UnstructuredGrid(topology, cell_types, meshio_mesh.geometry.x)
    return grid

# Mesh Creation

In [134]:
def volume_from_surface(mesh, mindihedral):
    tet = tetgen.TetGen(mesh)
    tet.tetrahedralize(order=1, mindihedral=mindihedral) 
    tetgen_mesh = tet.grid
    return tetgen_mesh

In [161]:
def create_tetgen_volume(obj_dir, mindihedral):
    mesh = pyvista.read(obj_dir)
    tetgen_mesh = volume_from_surface(mesh, mindihedral)
    return tetgen_mesh

def create_meshio_volume(obj_dir, mindihedral):
    tetgen_mesh = create_tetgen_volume(obj_dir, mindihedral)
    meshio_mesh = tetgen_to_meshio(tetgen_mesh)
    meshio.write('breast.xdmf', meshio_mesh)
    with dolfinx.io.XDMFFile(MPI.COMM_WORLD, 'breast.xdmf', 'r') as f:
        meshio_mesh = f.read_mesh(name="Grid")
    for ext in ('.h5', '.xdmf'):
        os.remove('breast'+ext)
    return meshio_mesh

In [136]:
# Example:
obj_dir = 'breast.obj'
mindihedral = 20
tetgen_mesh = create_tetgen_volume(obj_dir, mindihedral)
meshio_mesh = tetgen_to_meshio(tetgen_mesh)

# Mesh Visualisation

In [137]:
def get_submesh(tetgen_mesh, axis, cutoff):
    cell_center = tetgen_mesh.cell_centers().points
    mask = cell_center[:, axis] < cutoff
    cell_ind = mask.nonzero()[0]
    tetgen_submesh = tetgen_mesh.extract_cells(cell_ind)
    return tetgen_submesh

In [138]:
def visualise_mesh(tetgen_mesh, tetgen_submesh=None, title=None, show_grid=False):
    pyvista.start_xvfb(wait=0.05)
    p = pyvista.Plotter(notebook=True, window_size=[960,480]) #
    title = "Breast Mesh" if title is None else title
    p.add_text(title, name="title", position="upper_edge")
    if tetgen_submesh is not None:
        p.add_mesh(tetgen_mesh, style="wireframe", color="k") 
        p.add_mesh(tetgen_submesh, lighting=True, show_edges=True) 
    else: 
        p.add_mesh(tetgen_mesh, show_edges=True, edge_color='k', lighting=False)
    p.show_axes()
    if show_grid:
        p.show_grid()
    viewer = p.show(jupyter_backend='panel', return_viewer=True)
    return viewer

In [139]:
# Example:
tetgen_submesh = get_submesh(tetgen_mesh, axis=1, cutoff=80)
visualise_mesh(tetgen_mesh, tetgen_submesh, 
               title='Volume Mesh with Cross-Section',
               show_grid=True)

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)

# Gravity Rotation

In [177]:
# Using Euler angles - see https://www.autonomousrobotslab.com/frame-rotations-and-representations.html
# Here, y_rot = theta, x_rot = psi
ANGLE_TO_RAD = pi/180
def rotate_gravity(g_vector, y_rot, x_rot):
    # NB: Negative associated with y so increasing y_rot goesin 'right direction'
    theta, psi = -ANGLE_TO_RAD*y_rot, ANGLE_TO_RAD*x_rot
    rot_matrix = np.array([[         cos(theta),        0,          -sin(theta)],
                           [sin(psi)*sin(theta),  cos(psi), sin(psi)*cos(theta)],
                           [cos(psi)*sin(theta), -sin(psi), cos(psi)*cos(theta)]])
    rotated_g = rot_matrix @ g_vector
    return rotated_g

In [176]:
# Test:
g_vector = np.array([1,0,0])
y_rot = 90
x_rot = 0
rotate_gravity(g_vector, y_rot, x_rot)

array([ 0.,  0., -1.])

# Apply Load to Breast

In [158]:
def apply_loading(obj_dir, y_rot, x_rot, E, nu, rho, g, elem_order, num_steps, mindihedral):
    
    mesh = create_meshio_volume(obj_dir, mindihedral)
    V = dolfinx.VectorFunctionSpace(mesh, ("CG", elem_order))
    
    # Create lambda and mu fields:
    lambda_ = E*nu/((1+nu)*(1-2*nu))
    mu = E/(2*(1+nu))
    
    # Apply fixed BC:
    fixed = lambda x: x[0] < 10
    fixed_facets = dolfinx.mesh.locate_entities_boundary(mesh, mesh.topology.dim - 1, fixed)
    facet_tag = dolfinx.MeshTags(mesh, mesh.topology.dim-1, fixed_facets, 1)
    u_bc = dolfinx.Function(V)
    with u_bc.vector.localForm() as loc:
        loc.set(0)
    left_dofs = dolfinx.fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.indices[facet_tag.values==1])
    bcs = [dolfinx.DirichletBC(u_bc, left_dofs)]
    
    B = dolfinx.Constant(mesh, (0, 0, 0))
    T = dolfinx.Constant(mesh, (0, 0, 0))
    v = ufl.TestFunction(V)
    u = dolfinx.Function(V)

    d = len(u)
    I = ufl.variable(ufl.Identity(d))
    F = ufl.variable(I + ufl.grad(u))
    C = ufl.variable(F.T * F)
    Ic = ufl.variable(ufl.tr(C))
    J  = ufl.variable(ufl.det(F))
    psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lambda_ / 2) * (ufl.ln(J))**2
    P = ufl.diff(psi, F)
    
    metadata = {"quadrature_degree": elem_order}
    ds = ufl.Measure('ds', subdomain_data=facet_tag, metadata=metadata)
    dx = ufl.Measure("dx", metadata=metadata)
    F = ufl.inner(ufl.grad(v), P)*dx - ufl.inner(v, B)*dx - ufl.inner(v, T)*ds(2) 
    
    problem = dolfinx.fem.NonlinearProblem(F, u, bcs)
    solver = dolfinx.NewtonSolver(MPI.COMM_WORLD, problem)

    solver.atol = 1e-3
    solver.rtol = 1e-3
    solver.convergence_criterion = "incremental"
    
    g_vector = g*np.array([1,0,0])
    g_vector = rotate_gravity(g_vector, y_rot, x_rot)
    
    f_step = rho*g_vector/num_steps
    for n in range(num_steps):
        print(f"Performing load step {n+1}/{num_steps}")
        for i, f_i in enumerate(f_step):
            B.value[i] = (n+1)*f_i
        num_its, converged = solver.solve(u)
        assert(converged)
        u.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
    
    return (u, mesh)

# Visualise Breast Deformation

In [165]:
def plot_deformation(meshio_mesh, uh):
    mesh = meshio_mesh
    pyvista.start_xvfb(wait=0.05)
    topology, cell_types = dolfinx.plot.create_vtk_topology(mesh, mesh.topology.dim)
    grid = pyvista.UnstructuredGrid(topology, cell_types, mesh.geometry.x)
    
    p = pyvista.Plotter(notebook=True, window_size=[960,480]) #
    
    p.add_text("Deformed configuration", name="title", position="upper_edge")
    
    grid["u"] = uh.compute_point_values().real 
    actor_0 = p.add_mesh(grid, style="wireframe", color="k")
    warped = grid.warp_by_vector("u", factor=1.5)
    actor_1 = p.add_mesh(warped)
    
    p.show_axes()
    viewer = p.show(jupyter_backend='panel', return_viewer=True)
    return viewer

# Call Functions

In [184]:
# Fixed parameters:
elem_order = 2
W = 40 # in mm
L = 90 # in mm
nu = 0.33 # dimensionless
rho = 0.00102 # in g mm^-3
g = 9.81 # in m s^-2
num_steps = 5

# Variables:
y_rot = 90
x_rot = -90
E =  5 # in mPa

# Mesh information:
obj_dir = 'breast.obj'
mindihedral = 20

In [185]:
u, mesh = apply_loading(obj_dir, y_rot, x_rot, E, nu, rho, g, elem_order, num_steps, mindihedral)

Performing load step 1/5
Performing load step 2/5
Performing load step 3/5
Performing load step 4/5
Performing load step 5/5


In [186]:
plot_deformation(mesh, u)

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, color_mappers=[LinearColorMapper(id='106...], height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)